In [31]:
import pandas as pd

# Large dataset --> split into chunks of 1 200 000 lines per csv.
chunksize = 1_200_000
i = 0
output_dir = "csv_splits"

# The split csvs are saved into the defined output directory
os.makedirs(output_dir, exist_ok=True)

for chunk in pd.read_csv("out.csv", chunksize=chunksize, dtype={"column_name": str}):
    chunk.to_csv(os.path.join(output_dir, f"split_{i}.csv"), index=False)
    i += 1

print("Finished")

/tmp/ipykernel_4075/3388578945.py:11: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("out.csv", chunksize=chunksize, dtype={"column_name": str}):


Finished


In [32]:
import numpy as np
import glob
import os

def process_file(path, output_path):
    
    df = pd.read_csv(path, quotechar='"', low_memory=False)
    
    df.rename(columns=lambda x: x.strip(), inplace=True)

    df["licencePlate"] = (
        df["licencePlate"]
        .astype(str)
        .str.lower()
        .str.replace(" ", "", regex=False)
        .str.strip()
    )

    # Ensure proper ordering
    df["file_datetime"] = pd.to_datetime(df["file_datetime"], errors="coerce")
    df = df.sort_values(['licencePlate', 'file_datetime']).reset_index(drop=True)

    df["zipCode"] = (df["zipCode"].astype(str).str.extract(r'(\d{4})').astype(float).fillna(-1).astype(int))

    # Thresholds
    thr = 1e-3
    time_thr = pd.Timedelta(minutes=10)

    lat  = df['lat'].to_numpy()
    lon  = df['lon'].to_numpy()
    plate= df['licencePlate'].to_numpy()
    time = df['file_datetime'].to_numpy()

    # Plate change
    plate_change = np.r_[True, plate[1:] != plate[:-1]]

    # Location change
    move_change = np.r_[False,
        (np.abs(lat[1:] - lat[:-1]) > thr) |
        (np.abs(lon[1:] - lon[:-1]) > thr)
    ]

    # Time change
    time_diff = np.diff(time, prepend=time[0])
    time_change = time_diff > time_thr

    # Any boundary event
    boundary = plate_change | move_change | time_change
    b_idx = np.flatnonzero(boundary)

    # Closing segments
    end_idx   = np.r_[b_idx[1:] - 1, len(df) - 1]
    start_idx = b_idx[:len(end_idx)]

    # Build output
    grouped = pd.DataFrame({
        "licencePlate": df.loc[start_idx, "licencePlate"].to_numpy(),
        "start_time":   df.loc[start_idx, "file_datetime"].to_numpy(),
        "end_time":     df.loc[end_idx,   "file_datetime"].to_numpy(),
        "start_lat":    df.loc[start_idx, "lat"].to_numpy(),
        "start_lon":    df.loc[start_idx, "lon"].to_numpy(),
        "end_lat":      df.loc[end_idx,   "lat"].to_numpy(),
        "end_lon":      df.loc[end_idx,   "lon"].to_numpy(),
        "travel_time": np.round(
            (df.loc[end_idx, "file_datetime"].to_numpy() -
             df.loc[start_idx, "file_datetime"].to_numpy()) /
             np.timedelta64(1, "m")
        ).astype(int),
        "vehicleTypeId": df.loc[start_idx, "vehicleTypeId"].astype(str).to_numpy(),
        "zipCode": df.loc[start_idx, "zipCode"].to_numpy()
    })

    # car-type mapping
    car_types = {
        "1": {"model":"Renault Zoe","type":"car"},
        "2": {"model":"Renault Zoe","type":"car"},
        "6": {"model":"unknown","type":"van"},
        "9":  {"model":"Renault Zoe","type":"car"},
        "10": {"model":"Renault Zoe","type":"car"},
        "14": {"model":"Renault Zoe","type":"car"},
        "25": {"model":"unknown","type":"van"},
        "26": {"model":"Renault Zoe","type":"car"},
        "31": {"model":"unknown","type":"van"},
        "32": {"model":"SAIC Motor MAXUS E-Deliver 3","type":"van"},
        "34": {"model":"Renault Zoe","type":"car"},
        "35": {"model":"Renault Zoe","type":"car"},
        "57": {"model":"Renault Zoe","type":"car"},
        "64": {"model":"Renault Zoe","type":"car"},
        "74": {"model":"unknown","type":"van"},
        "76": {"model":"Mercedes eVito","type":"van"},
        "77": {"model":"Renault Zoe","type":"car"},
        "79": {"model":"Peugeot e-Partner","type":"van"},
        "86": {"model":"Renault Zoe","type":"car"},
        "91": {"model":"Renault Zoe","type":"car"},
        "94": {"model":"Renault Zoe","type":"car"},
        "95": {"model":"Renault Kangoo","type":"van"},
        "96": {"model":"Renault Zoe","type":"car"},
        "97": {"model":"Renault Zoe","type":"car"},
        "99": {"model":"Renault Zoe","type":"car"},
        "102": {"model":"Renault Megane","type":"car"},
        "103": {"model":"Opel Vivaro Electric","type":"van"},
        "105": {"model":"Renault Zoe","type":"car"},
        "106": {"model":"Renault Zoe","type":"car"},
        "107": {"model":"Renault Zoe","type":"car"},
        "109": {"model":"Renault Trafic E-Tech","type":"van"},
        "111": {"model":"Ford E-Transit","type":"van"}
    }

    df_car_types = pd.DataFrame(car_types).T
    df_car_types.index.name = "vehicleTypeId"
    df_car_types.reset_index(inplace=True)
    
    grouped["vehicleTypeId"] = grouped["vehicleTypeId"].astype(str)
    grouped = grouped.merge(df_car_types, on="vehicleTypeId", how="left")


    # Filling NaN with 'unknown'
    grouped["model"] = grouped["model"].fillna("unknown")
    grouped["type"] = grouped["type"].fillna("unknown")


    # Postal-code mapping
    post_codes = {
        1: {"name": "Bronshoj",         "zip_from": 2700, "zip_to": 2700, "setting": 2700},
        2: {"name": "Kobenhavn K",      "zip_from": 1050, "zip_to": 1473, "setting": 1050},
        3: {"name": "Kobenhavn N",      "zip_from": 2200, "zip_to": 2200, "setting": 2200},
        4: {"name": "Kobenhavn NV",     "zip_from": 2400, "zip_to": 2400, "setting": 2400},
        5: {"name": "Kobenhavn O",      "zip_from": 2100, "zip_to": 2100, "setting": 2100},
        6: {"name": "Kobenhavn S",      "zip_from": 2300, "zip_to": 2300, "setting": 2300},
        7: {"name": "Kobenhavn SV",     "zip_from": 2450, "zip_to": 2450, "setting": 2450},
        8: {"name": "Kobenhavn V",      "zip_from": 1550, "zip_to": 1799, "setting": 1550},
        9: {"name": "Nordhavn",         "zip_from": 2150, "zip_to": 2150, "setting": 2150},
        10: {"name": "Valby",           "zip_from": 2500, "zip_to": 2500, "setting": 2500},
        11: {"name": "Vanlose",         "zip_from": 2720, "zip_to": 2720, "setting": 2720},
        12: {"name": "Frederiksberg C", "zip_from": 1800, "zip_to": 2000, "setting": 2000},
    }
    df_codes = pd.DataFrame(post_codes).T
    df_codes['zip_from'] = df_codes['zip_from'].astype(int)
    df_codes['zip_to'] = df_codes['zip_to'].astype(int)
    
    # Convert zipCode to numeric safely
    grouped['zipCode'] = pd.to_numeric(grouped['zipCode'], errors='coerce').fillna(-1).astype(int)

    # Interval index for mapping
    iv = pd.IntervalIndex.from_arrays(df_codes['zip_from'], df_codes['zip_to'], closed='both')

    zips = grouped['zipCode'].to_numpy()
    pos = iv.get_indexer(zips)
    mask = pos != -1
    zip_fixed = zips.copy()
    zip_fixed[mask] = df_codes['setting'].to_numpy()[pos[mask]]
    grouped['postcode'] = zip_fixed
    grouped['zipCodeFixed'] = zip_fixed

    grouped["end_zipCode"] = df.loc[end_idx, "zipCode"].to_numpy()
    grouped["end_zipCode"] = pd.to_numeric(df.loc[end_idx, "zipCode"], errors='coerce').fillna(-1).astype(int)

    z2 = grouped["end_zipCode"].to_numpy()
    pos2 = iv.get_indexer(z2)
    mask2 = pos2 != -1
    zip_fixed2 = z2.copy()
    zip_fixed2[mask2] = df_codes['setting'].to_numpy()[pos2[mask2]]

    grouped["end_postcode"] = zip_fixed2

    zip_to_name = df_codes.set_index("setting")["name"].to_dict()

    valid_zipcodes = set(df_codes["setting"].astype(int))

    mask_keep = (
        grouped["zipCodeFixed"].isin(valid_zipcodes)
        & grouped["end_zipCode"].isin(valid_zipcodes)
    )
    
    grouped = grouped[mask_keep].reset_index(drop=True)


    grouped["start_area"] = grouped["postcode"].map(zip_to_name).fillna("Unknown")
    grouped["end_area"] = grouped["end_postcode"].map(zip_to_name).fillna("Unknown")

    grouped["route"] = grouped["start_area"] + " → " + grouped["end_area"]

    # Temporal fields
    grouped["day_of_week"] = pd.to_datetime(grouped["start_time"]).dt.day_name()
    grouped["hour_of_day"] = pd.to_datetime(grouped["start_time"]).dt.hour

    grouped = grouped[
        ~(
            (grouped["travel_time"] == 0) &
            (grouped["start_lat"] == grouped["end_lat"]) &
            (grouped["start_lon"] == grouped["end_lon"])
        )
    ]

    # Safe write (overwrite)
    temp = output_path + ".tmp"
    grouped.to_csv(temp, index=False)
    os.replace(temp, output_path)

In [33]:
# Running the process_file function
output_dir = "csv_splits"

input_files = sorted(glob.glob(os.path.join(output_dir, "split_*.csv")))

for f in input_files:
    process_file(f, f)


In [34]:
# Gathering the processed split csv into one complete file
output_dir = "csv_splits"

processed_files = sorted(glob.glob(os.path.join(output_dir, "split_*.csv")))
output_final = "transformed_output.csv"

dfs = []
for f in processed_files:
    df = pd.read_csv(
        f,
        quotechar='"',
        parse_dates=["start_time", "end_time"],
        low_memory=False
    )
    dfs.append(df)

final_df = pd.concat(dfs, ignore_index=True)

final_df.to_csv(output_final, index=False)

print("Final output written to:", output_final)


Final output written to: transformed_output.csv
